In [1]:
import tkinter as tk
from tkinter import ttk
import cv2
import numpy as np
import math

image = None
grayLevel = 0
img_label = None  # Global variable to store the image label
photo_image = None  # Global variable to store the PhotoImage

def highLevelMapping():
    global image, grayLevel
    ranges = 256 / grayLevel
    power = int(math.log2(ranges))
    or_mask = (2 ** power) - 1
    or_result = cv2.bitwise_or(image, or_mask)
    show_result_image(or_result)

def lowLevelMapping():
    global image, grayLevel
    ranges = 256 / grayLevel
    power = int(math.log2(ranges))
    and_mask = int('1' * (8 - power) + '0' * power, 2)
    and_result = cv2.bitwise_and(image, and_mask)
    show_result_image(and_result)

def lowAvgMapping():
    global image, grayLevel
    ranges = 256 / grayLevel
    power = int(math.log2(ranges))
    and_mask = int('1' * (8 - power) + '0' * power, 2)
    or_avg_mask = (2 ** (power - 1)) - 1
    and_result = cv2.bitwise_and(image, and_mask)
    or_avg_result = cv2.bitwise_or(and_result, or_avg_mask)
    show_result_image(or_avg_result)

def highAvgMapping():
    global image, grayLevel
    ranges = 256 / grayLevel
    power = int(math.log2(ranges))
    or_mask = (2 ** power) - 1
    or_result = cv2.bitwise_or(image, or_mask)
    and_avg_mask = int('1' * (8 - power + 1) + '0' * (power - 1), 2)
    and_avg_result = cv2.bitwise_and(or_result, and_avg_mask)
    show_result_image(and_avg_result)

def show_result_image(image):
    cv2.imshow("Result", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def open_image():
    global image, photo_image
    image = cv2.imread('img.jpg', cv2.IMREAD_GRAYSCALE)

    if image is not None:
        if photo_image is not None:
            img_label.config(image="")
            img_label.img = None
            photo_image = None

        photo_image = tk.PhotoImage(data=cv2.imencode('.png', image)[1].tobytes())
        img_label.config(image=photo_image)
        img_label.img = photo_image

def perform_action():
    selected_option = action_var.get()
    if selected_option == "High-Level Mapping":
        highLevelMapping()
    elif selected_option == "Low-Level Mapping":
        lowLevelMapping()
    elif selected_option == "Low-Avg Mapping":
        lowAvgMapping()
    elif selected_option == "High-Avg Mapping":
        highAvgMapping()

def set_quantization_level(event):
    global grayLevel
    selected_quantization = quantization_level_var.get()
    if selected_quantization == "Select Action":
        grayLevel = 0
    else:
        grayLevel = int(selected_quantization)

root = tk.Tk()
root.title("Quantization Assignment")

img_label = tk.Label(root)
img_label.pack()

open_image()

action_var = tk.StringVar()
action_var.set("Select Action")
action_dropdown = ttk.Combobox(root, textvariable=action_var, values=[
    "High-Level Mapping", "Low-Level Mapping", "Low-Avg Mapping", "High-Avg Mapping"])
action_dropdown.pack()

quantization_label = tk.Label(root, text="Quantization Level:")
quantization_label.pack()

quantization_level_var = tk.StringVar()
quantization_level_var.set("Select Action")
quantization_level_dropdown = ttk.Combobox(root, textvariable=quantization_level_var, values=["Select Action", "2", "4", "8", "16", "32", "64", "128"])
quantization_level_dropdown.pack()
quantization_level_dropdown.bind("<<ComboboxSelected>>", set_quantization_level)

perform_action_button = tk.Button(root, text="Perform Action", command=perform_action)
perform_action_button.pack()

root.mainloop()
